# Imports

In [3]:
import pandas as pd
import numpy as np
import ast

# Load evaluations 10k

In [6]:
df_all_seeds = pd.DataFrame()
for seed in [123, 1234, 12345, 123456, 1234567]:
    for private_agent in [0, 1]:
        for collective_agent in [0, 1]:
            file_path = f"./seed_{seed}/tragedy_test_Pa{private_agent}_Ca{collective_agent}/evaluation.csv"
            df = pd.read_csv(file_path)
            df.drop(columns=["Unnamed: 0","background_per_capita_return","background_player_names","background_player_returns"], inplace=True)
            # pp private vs private; cc collective vs collective; pc private vs collective
            df["focal_player_names"] = df["focal_player_names"].apply(lambda x: ast.literal_eval(x))
            # agent_0 is the private agent and agent_1 is the collective agent (in all evaluations)
            df['player0'] = df['focal_player_names'].apply(lambda x: "private" if x[0]=="agent_0" else "collective")
            df['player1'] = df['focal_player_names'].apply(lambda x: "private" if x[1]=="agent_0" else "collective")
            # NOTE: remember that cp == pc
            df['evaluation_scenario'] = df['player0'].str[0] + df['player1'].str[0]
            # move focal players returns to separate columns
            df['player0_score'] = df['focal_player_returns'].apply(lambda x: np.fromstring(x[1:-1], sep=' ')[0])
            df['player1_score'] = df['focal_player_returns'].apply(lambda x: np.fromstring(x[1:-1], sep=' ')[1])
            df['mean_score'] = df[['player0_score', 'player1_score']].mean(axis=1)
            # drop unnecessary columns
            df.drop(columns=['focal_player_returns', 'focal_player_names', 'focal_per_capita_return'], inplace=True)
            # add seed and agents used (the number of agent refers to the training process - each train session yields 2 agents)
            df['seed'] = seed
            df['private_agent'] = f"agent_{private_agent}"
            df['collective_agent'] = f"agent_{collective_agent}"
            # reorder columns
            df = df[['seed', 'private_agent', 'collective_agent', 'player0', 'player1', 'evaluation_scenario', 'player0_score', 'player1_score', 'mean_score']]
            # add column private_agent_score 
            df['private_agent_score'] = df.apply(lambda row: row['player0_score'] if (row['player0'] == 'private' and row['player1'] != 'private') else (row['player1_score'] if (row['player1'] == 'private' and row['player0'] != 'private') else None), axis=1)
            # add column collective_agent_score
            df['collective_agent_score'] = df.apply(lambda row: row['player0_score'] if (row['player0'] == 'collective' and row['player1'] != 'collective') else (row['player1_score'] if (row['player1'] == 'collective' and row['player0'] != 'collective') else None), axis=1)
            # concatenate to all seeds
            df_all_seeds = pd.concat([df_all_seeds, df])
            # reset index
            df_all_seeds.reset_index(drop=True, inplace=True)
df_all_seeds.to_csv("10k_iter_all_seeds_evaluation.csv", index=False)
df_all_seeds

,seed,private_agent,collective_agent,player0,player1,evaluation_scenario,player0_score,player1_score,mean_score,private_agent_score,collective_agent_score
0,123,agent_0,agent_0,private,collective,pc,13.0,17.0,15.0,13.0,17.0
1,123,agent_0,agent_0,private,collective,pc,8.0,6.0,7.0,8.0,6.0
2,123,agent_0,agent_0,collective,private,cp,2.0,4.0,3.0,4.0,2.0
3,123,agent_0,agent_0,private,collective,pc,1.0,4.0,2.5,1.0,4.0
4,123,agent_0,agent_0,private,private,pp,24.0,23.0,23.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9995,1234567,agent_1,agent_1,private,collective,pc,7.0,3.0,5.0,7.0,3.0
9996,1234567,agent_1,agent_1,private,collective,pc,1.0,4.0,2.5,1.0,4.0
9997,1234567,agent_1,agent_1,private,private,pp,3.0,2.0,2.5,NaN,NaN
9998,1234567,agent_1,agent_1,collective,collective,cc,25.0,24.0,24.5,NaN,NaN
